<a href="https://colab.research.google.com/github/mbubesa/ML-Algo-Exploration/blob/main/unet_prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import modules

In [1]:
import torch
from torch import nn
import torchvision.transforms.functional

In [2]:
class DoubleConvolution(nn.Module):
  def __init__(self, in_channels: int, out_channels: int):
    super().__init__()
    self.first = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
    self.act1 = nn.ReLU()

    self.second = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
    self.act2 = nn.ReLU()

    def forward(self, x: torch.tensor):
      x = self.first(x)
      x = self.act1(x)

      x = self.second(x)
      return self.act2(x)

In [3]:
print("Leo is the greatest person living")

Leo is the greatest person living


# Down Sampling

In [5]:
class DownSample(nn.Module):
  def __init__(self):
    super().__init__()

    self.pool = nn.MaxPool2d(2)

  def forward(self, x: torch.tensor):
    return self.pool(x)

# Upsampling

In [7]:
class UpSample(nn.Module):
  def __init__(self):
    super().__init__()
    self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, strides=2)

  def forward(self, x: torch.tensor):
    return self.up(x)

In [11]:
class CropAndConcat(nn.Module):
  def __init__(self, x: torch.tensor, contracting_x: torch.tensor):
    contracting_x = torchvision.transforms.functional.center_crop(contracting_x, [x.shape[2], x.shape[3]])
    x = torch.cat([x, contracting_x], dim=1)
    return x

# The Unet

In [12]:
class Unet(nn.Module):
  def __init__(self, in_channels: int, out_channels: int):
    super().__init__()

    self.down_conv = nn.ModuleList([DoubleConvolution(i, o) for i, o in
                                  [(in_channels, 64), (64, 128), (128, 256), (256, 512)]])
    self.down_sample = nn.ModuleList([DownSample() for _ in range(4)])
    self.down_sample = nn.ModuleList([DownSample() for _ in range(4)])
    self.up_sample = nn.ModuleList([UpSample(i, o) for i, o in
                                  [(1024, 512), (512, 256), (256, 128), (128, 64)]])
    self.up_conv = nn.ModuleList([DoubleConvolution(i, o) for i, o in
                                 [(1024, 512), (512, 256), (256, 128), (128, 64)]])
    self.concat = nn.ModuleList([CropAndConcat() for _ in range(4)])
    self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

  def forward(self, x: torch.Tensor):
    pass_through = []
    for i in range(len(self.down_conv)):
      x = self.down_conv[i](x)
      pass_through.append(x)
      x = self.down_sample[i](x)

    x = self.middle_conv(x)
    for i in range(len(self.up_conv)):
      x = self.up_sample[i](x)
      x = self.concat[i](x, pass_through.pop())
      x = self.up_conv[i](x)

    x = self.final_conv(x)
    return x